In [2]:
#from keras.models import Sequential
#from keras.layers import Dense
import numpy
#from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import KFold
#from keras.callbacks import EarlyStopping

#from keras.models import load_model
import math

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from scipy.stats import kruskal

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
import math
from sklearn.metrics import accuracy_score
# fix random seed for reproducibility
numpy.random.seed(7)

import warnings
warnings.filterwarnings("ignore")
global Best_Acc
Best_Acc=0

In [3]:
#Importing the Train Data 
import pandas as pd
df = pd.read_csv("F:\Parkinsons Disease PD/train_data.txt")
X_first = numpy.array(df)
X_train = numpy.delete(X_first, 27, 1) #Removing UPDRS Column
X_withlabels = numpy.delete(X_train, 0,1) #Removing Subject ID column
print("X_with contains 26 input features and class label but not with patient id", X_withlabels.shape)               
X = numpy.delete(X_withlabels, 26, 1) #Removing Label Column
print("X contains 26 input features only and there is no output label", X.shape)
Y = X_withlabels[: ,26]
print(Y)

X_with contains 26 input features and class label but not with patient id (1040, 27)
X contains 26 input features only and there is no output label (1040, 26)
[1. 1. 1. ... 0. 0. 0.]


In [4]:
print(X.shape)
print(Y.shape)
print(Y)

(1040, 26)
(1040,)
[1. 1. 1. ... 0. 0. 0.]


In [5]:
#(Considering Training Database of the Dataset)
i=2
indices = numpy.array(range(0, 2))
lb = 26
ub = 28
while i<=40:
    indices = numpy.append(indices, range(lb, ub))    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    lb = lb+26
    ub = ub+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y_vowel = X_train_withlabels[:, 26]
X_vowel = numpy.delete(X_train_withlabels, 26, 1)
print(X_vowel.shape)
print(Y_vowel.shape)

(80,)
(80, 27)
(80, 26)
(80,)


In [6]:
print(Y)

[1. 1. 1. ... 0. 0. 0.]


# Seperating X for PD and Healthy gropugs of Training Database

In [7]:
X_PD_Train = X_vowel[range(0, 40)]
X_H_Train = X_vowel[range(40, 80)]
print(X_PD_Train.shape)
print(X_H_Train.shape)

(40, 26)
(40, 26)


# Mean and Std of Each Feature of PD group (Training Database)

In [8]:
from scipy import stats
J = range(0, 26)
for j in J:
    mean_f = numpy.mean(X_PD_Train[:, j])
    std_d = numpy.std(X_PD_Train[:, j])
    print("feautre no =", j, "   and mean ====", mean_f, "  and std ===", std_d)

feautre no = 0    and mean ==== 0.7936249999999999   and std === 0.6625059881804843
feautre no = 1    and mean ==== 5.0973275e-05   and std === 4.2999924737717566e-05
feautre no = 2    and mean ==== 0.42140000000000005   and std === 0.4162117129538764
feautre no = 3    and mean ==== 0.42917500000000003   and std === 0.3728058400494821
feautre no = 4    and mean ==== 1.2641749999999998   and std === 1.2487681908084463
feautre no = 5    and mean ==== 7.185275   and std === 3.89568327118299
feautre no = 6    and mean ==== 0.6819   and std === 0.340339815478589
feautre no = 7    and mean ==== 3.531325   and std === 2.0980747768787937
feautre no = 8    and mean ==== 4.344125   and std === 2.5052980879278617
feautre no = 9    and mean ==== 6.2353250000000005   and std === 3.388343130406807
feautre no = 10    and mean ==== 10.594325000000001   and std === 6.294195796078718
feautre no = 11    and mean ==== 0.9478628   and std === 0.05192140985768782
feautre no = 12    and mean ==== 0.066191599

# Mean and Std of Each Feature of H group (Training Database)

In [9]:
from scipy import stats
J = range(0, 26)
for j in J:
    mean_f = numpy.mean(X_H_Train[:, j])
    std_d = numpy.std(X_H_Train[:, j])
    print("feautre no =", j, "   and mean ====", mean_f, "  and std ===", std_d)

feautre no = 0    and mean ==== 0.7727499999999999   and std === 0.7559899387558012
feautre no = 1    and mean ==== 5.020867499999999e-05   and std === 5.203689981560561e-05
feautre no = 2    and mean ==== 0.392325   and std === 0.4666082611516859
feautre no = 3    and mean ==== 0.36825   and std === 0.3403663283875184
feautre no = 4    and mean ==== 1.1768750000000001   and std === 1.399809668981823
feautre no = 5    and mean ==== 6.853425   and std === 5.480496190526456
feautre no = 6    and mean ==== 0.6283749999999999   and std === 0.45200042519338407
feautre no = 7    and mean ==== 3.5422250000000006   and std === 3.092133498795775
feautre no = 8    and mean ==== 4.363325   and std === 4.0666197288872485
feautre no = 9    and mean ==== 5.3486   and std === 3.787156834618815
feautre no = 10    and mean ==== 10.62675   and std === 9.276505648006689
feautre no = 11    and mean ==== 0.937588725   and std === 0.08978042436243758
feautre no = 12    and mean ==== 0.092647175   and std ==

# Mean and Std of Each Feature of PD group (Testing Database)

In [10]:
#Importing the Test Data 
df = pd.read_csv("E:\PD Dataset From Laptop/test_data.txt")
X_first = numpy.array(df)
X_test_withlabels = numpy.delete(X_first, 0,1)
X_test = numpy.delete(X_test_withlabels, 26, 1)
print("X_with contains 26 input features and class label but not with patient id", X_test_withlabels.shape)   
print(X_test.shape)
Y_test = X_test_withlabels[:, 26]
print(Y_test.shape)

X_with contains 26 input features and class label but not with patient id (168, 27)
(168, 26)
(168,)


In [11]:
from scipy import stats
J = range(0, 26)
for j in J:
    mean_f = numpy.mean(X_test[:, j])
    std_d = numpy.std(X_test[:, j])
    print("feautre no =", j, "   and mean ====", mean_f, "  and std ===", std_d)

feautre no = 0    and mean ==== 0.6406845238095238   and std === 0.8219573484175625
feautre no = 1    and mean ==== 4.2532142857142855e-05   and std === 5.5119339490639335e-05
feautre no = 2    and mean ==== 0.33540476190476187   and std === 0.44877461781652916
feautre no = 3    and mean ==== 0.37489285714285714   and std === 0.526804393938385
feautre no = 4    and mean ==== 1.006220238095238   and std === 1.3463273076604827
feautre no = 5    and mean ==== 5.162654761904762   and std === 3.0603054819274855
feautre no = 6    and mean ==== 0.4743452380952381   and std === 0.2657218602244343
feautre no = 7    and mean ==== 2.679363095238095   and std === 1.7942841304489363
feautre no = 8    and mean ==== 3.044815476190476   and std === 1.6750731492196627
feautre no = 9    and mean ==== 4.2102155690476195   and std === 2.0365289831094393
feautre no = 10    and mean ==== 8.038089285714285   and std === 5.382765080759805
feautre no = 11    and mean ==== 0.9626099047619048   and std === 0.048

# Combine P values across all the three goups

In [12]:
J = range(0, 26)
for j in J:
    data1 = X_H_Train[:, j]
    data2 = X_test[:, j]
    data3 = X_PD_Train[:, j]
    stat, p = kruskal(data1, data2, data3)
    print("feautre no =", j, "   P value ====", p)

feautre no = 0    P value ==== 0.00031600889055685434
feautre no = 1    P value ==== 0.009070010600680705
feautre no = 2    P value ==== 0.007174164225483645
feautre no = 3    P value ==== 0.005813441916341583
feautre no = 4    P value ==== 0.007153151658888425
feautre no = 5    P value ==== 0.006005561726118254
feautre no = 6    P value ==== 0.0009916541438595055
feautre no = 7    P value ==== 0.03947123021695291
feautre no = 8    P value ==== 0.007262216370493356
feautre no = 9    P value ==== 0.0014935897601650282
feautre no = 10    P value ==== 0.039471380774429715
feautre no = 11    P value ==== 0.0013794854862141514
feautre no = 12    P value ==== 0.0003181299499321334
feautre no = 13    P value ==== 0.01346190532440859
feautre no = 14    P value ==== 0.9683149057478564
feautre no = 15    P value ==== 0.9289661203709598
feautre no = 16    P value ==== 1.2528098507414338e-07
feautre no = 17    P value ==== 0.8725405684626488
feautre no = 18    P value ==== 0.05049079932299938
feau